<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_classification_no_pretraining_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Things to change in real run
* Use the proper datasets (not dev for validation?)




# Setup

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

Mounted at /content/drive
/content/drive/MyDrive/AdaptOrDie


In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [3]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00


In [4]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")
    return gpu_used

print_gpu_utilization()

GPU 448 MB


448

In [5]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_helpfulness"
train_1 = load_dataset(dataset_name, split="train[:5%]")
train_2 = load_dataset(dataset_name, split="train[5%:10%]")
# train_3 = load_dataset(dataset_name, split="train[10%:20%]")
# train_4 = load_dataset(dataset_name, split="train[20%:40%]")
# train_5 = load_dataset(dataset_name, split="train[40%:]")
validation_dataset = load_dataset(dataset_name, split="dev")

train_datasets = [train_1, train_2]
# train_datasets = [train_1, train_2, train_3, train_4, train_5]

print(train_1)
print(train_2)
# print(train_3)
# print(train_4)
# print(train_5)
print(validation_dataset)

print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5763
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5762
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})
GPU 448 MB


448

In [6]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU 448 MB


448

In [7]:
adapter_hub_name = "adapter_HUB_classification_no_pretraining_test"
adapter_name = "adapter_classification_seq_bn_no_pretraining"
adapter_type = "seq_bn" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for masked language modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"}
  )

# Set the adapter to be used for training
model.train_adapter(adapter_name)


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


448

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
!pip install scikit-learn

In [11]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Train

In [12]:
from transformers import TrainingArguments
from adapters import AdapterTrainer

all_results = []

In [13]:
for i, train_dataset in enumerate(train_datasets):
    saved_results = {}

    print(f"Training dataset {i}")
    training_args = TrainingArguments(
        output_dir=f"adapter_classifier_training_output_{i}",
        learning_rate=1e-4,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_steps=100,
        overwrite_output_dir=True,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
    )

    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    training_results = trainer.train()
    saved_results["train"] = training_results
    print(training_results)

    eval_results = trainer.evaluate()
    saved_results["eval"] = eval_results
    print(eval_results)

    saved_results['gpu'] = print_gpu_utilization()

    all_results.append(saved_results)


Training dataset 0


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,0.429100
200,0.380100
300,0.365600
400,0.366100
500,0.354700
600,0.328700
700,0.330500
800,0.313900
900,0.317700
1000,0.310900


Checkpoint destination directory adapter_classifier_training_output_0/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory adapter_classifier_training_output_0/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1083, training_loss=0.3498042803171709, metrics={'train_runtime': 318.623, 'train_samples_per_second': 54.262, 'train_steps_per_second': 3.399, 'total_flos': 4359775986565506.0, 'train_loss': 0.3498042803171709, 'epoch': 3.0})


{'eval_loss': 0.34481820464134216, 'eval_accuracy': 0.8592, 'eval_f1_macro': 0.5395348569497009, 'eval_runtime': 38.0089, 'eval_samples_per_second': 131.548, 'eval_steps_per_second': 8.235, 'epoch': 3.0}
GPU 9975 MB
Training dataset 1


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Step,Training Loss
100,0.371400
200,0.361800
300,0.344400
400,0.386800
500,0.326100
600,0.329200
700,0.355500
800,0.377900
900,0.316000
1000,0.328600


TrainOutput(global_step=1083, training_loss=0.34796997098491245, metrics={'train_runtime': 280.1484, 'train_samples_per_second': 61.703, 'train_steps_per_second': 3.866, 'total_flos': 4360701317394012.0, 'train_loss': 0.34796997098491245, 'epoch': 3.0})


{'eval_loss': 0.33721137046813965, 'eval_accuracy': 0.8618, 'eval_f1_macro': 0.5708240340031584, 'eval_runtime': 37.9379, 'eval_samples_per_second': 131.794, 'eval_steps_per_second': 8.25, 'epoch': 3.0}
GPU 13857 MB


In [27]:
all_results[0]["train"]
all_results[0]["train"][1]
all_results[0]["train"][2]["train_runtime"]
all_results[0]["eval"]["eval_loss"]

0.34481820464134216

In [28]:
for result in all_results:
  training_loss = result["train"][1]
  training_time = result["train"][2]["train_runtime"]
  eval_loss = result["eval"]["eval_loss"]
  eval_f1_macro = 100.0 * result["eval"]["eval_f1_macro"]
  eval_accuracy = 100.0 * result["eval"]["eval_accuracy"]
  print(f"{i}: Eval F1: {eval_f1_macro:.3f} Acc: {eval_accuracy:.3f} Loss: {eval_loss:.5f} || Training Loss: {training_loss:.4f} Time: {training_time:.0f}")

1: Eval F1: 53.953 Acc: 85.920 Loss: 0.34482 || Training Loss: 0.3498 Time: 319
1: Eval F1: 57.082 Acc: 86.180 Loss: 0.33721 || Training Loss: 0.3480 Time: 280


In [29]:
trainer.state.log_history

[{'loss': 0.3714,
  'learning_rate': 9.076638965835642e-05,
  'epoch': 0.28,
  'step': 100},
 {'loss': 0.3618,
  'learning_rate': 8.153277931671284e-05,
  'epoch': 0.55,
  'step': 200},
 {'loss': 0.3444,
  'learning_rate': 7.229916897506925e-05,
  'epoch': 0.83,
  'step': 300},
 {'loss': 0.3868,
  'learning_rate': 6.306555863342567e-05,
  'epoch': 1.11,
  'step': 400},
 {'loss': 0.3261,
  'learning_rate': 5.383194829178209e-05,
  'epoch': 1.39,
  'step': 500},
 {'loss': 0.3292,
  'learning_rate': 4.459833795013851e-05,
  'epoch': 1.66,
  'step': 600},
 {'loss': 0.3555,
  'learning_rate': 3.536472760849492e-05,
  'epoch': 1.94,
  'step': 700},
 {'loss': 0.3779,
  'learning_rate': 2.613111726685134e-05,
  'epoch': 2.22,
  'step': 800},
 {'loss': 0.316,
  'learning_rate': 1.6897506925207758e-05,
  'epoch': 2.49,
  'step': 900},
 {'loss': 0.3286,
  'learning_rate': 7.663896583564174e-06,
  'epoch': 2.77,
  'step': 1000},
 {'train_runtime': 280.1484,
  'train_samples_per_second': 61.703,
  

In [30]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

,loss,learning_rate,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss,eval_loss,eval_accuracy,eval_f1_macro,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.3714,0.000091,0.28,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.3618,0.000082,0.55,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.3444,0.000072,0.83,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3868,0.000063,1.11,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.3261,0.000054,1.39,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.3292,0.000045,1.66,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.3555,0.000035,1.94,700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.3779,0.000026,2.22,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.3160,0.000017,2.49,900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.3286,0.000008,2.77,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/adapter_HUB_classification_no_pretraining_test/commit/1d56448dfd2bd6320012f666f0e94582443214cd', commit_message='Upload model', commit_description='', oid='1d56448dfd2bd6320012f666f0e94582443214cd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# print("Disconnecting Session")
# from google.colab import runtime
# runtime.unassign()